In [ ]:
# !pip install pandas 
# !pip install googletrans==4.0.0-rc1
# ! pip install deep-translator

In [15]:
import pandas as pd

# โหลดข้อมูลจากไฟล์ CSV
df = pd.read_csv('../dataset/dataset.csv')

# ตรวจสอบข้อมูลเบื้องต้น
# print(df.head())
print(df['label'].value_counts())

label
minimum     2587
moderate     394
mild         290
severe       282
Name: count, dtype: int64


Data Augmentation(Back Translation)

In [1]:
from deep_translator import GoogleTranslator
import pandas as pd

# ฟังก์ชันสำหรับ Back Translation โดยใช้ภาษาไทยเป็นภาษากลาง
def back_translate(text, src='en', mid='th'):
    try:
        print(f"Original: {text}")  # แสดงข้อความก่อนแปล
        translated = GoogleTranslator(source=src, target=mid).translate(text)  # แปลเป็นภาษาไทย
        print(f"Translated to Thai: {translated}")  # แสดงข้อความหลังแปลเป็นไทย
        back_translated = GoogleTranslator(source=mid, target=src).translate(translated)  # แปลกลับเป็นอังกฤษ
        print(f"Back Translated to English: {back_translated}\n")  # แสดงข้อความหลังแปลกลับ
        return back_translated
    except Exception as e:
        print(f"Error translating: {e}")
        return text  # ถ้าแปลไม่ได้ให้คืนค่าเดิม

# โหลดข้อมูล
print("Loading dataset...")
df = pd.read_csv('../dataset/dataset.csv')
print(f"Dataset loaded: {df.shape[0]} rows\n")

# เลือกเฉพาะ class ที่มีน้อย (mild, moderate, severe) มาเพิ่มข้อมูล
print("Filtering underrepresented classes (mild, moderate, severe)...")
df_aug = df[df['label'].isin(['mild', 'moderate', 'severe'])].copy()
print(f"Selected {df_aug.shape[0]} rows for augmentation\n")

# ใช้ Back Translation
print("Applying Back Translation...")
df_aug['text'] = df_aug['text'].apply(lambda x: back_translate(x))

# รวมข้อมูลใหม่กับข้อมูลเดิม
df = pd.concat([df, df_aug])
print(f"New dataset size after augmentation: {df.shape[0]} rows\n")

# บันทึกข้อมูลที่เพิ่มแล้ว
output_file = 'dataset_augmented.csv'
df.to_csv(output_file, index=False)
print(f"Augmented dataset saved as {output_file}")


Loading dataset...
Dataset loaded: 3553 rows

Filtering underrepresented classes (mild, moderate, severe)...
Selected 966 rows for augmentation

Applying Back Translation...
Original: He said he had not felt that way before, suggeted I go rest and so ..TRIGGER AHEAD IF YOUI'RE A HYPOCONDRIAC LIKE ME: i decide to look up "feelings of doom" in hopes of maybe getting sucked into some rabbit hole of ludicrous conspiracy, a stupid "are you psychic" test or new age b.s., something I could even laugh at down the road. No, I ended up reading that this sense of doom can be indicative of various health ailments; one of which I am prone to.. So on top of my "doom" to my gloom..I am now f'n worried about my heart. I do happen to have a physical in 48 hours.
Translated to Thai: เขาบอกว่าเขาไม่เคยรู้สึกแบบนั้นมาก่อนฉันไปพักผ่อนและ .. ทริกเกอร์ไปข้างหน้าถ้าคุณเป็นคน hypocondriac อย่างฉัน: ฉันตัดสินใจที่จะค้นหา "ความรู้สึกของการลงโทษ" ด้วยความหวังว่าอาจจะถูกดูดเข้าไปในหลุมกระต่าย การสมรู้ร่วมคิดที่น่า

In [1]:
import pandas as pd

# โหลดข้อมูลจากไฟล์ CSV
df01 = pd.read_csv('../dataset/dataset.csv')
df02 = pd.read_csv('../dataset/dataset_augmented1.csv')

# ตรวจสอบข้อมูลเบื้องต้น
print(df01['label'].value_counts())
print(df02['label'].value_counts())

label
minimum     2587
moderate     394
mild         290
severe       282
Name: count, dtype: int64
label
minimum     2587
moderate     788
mild         580
severe       564
Name: count, dtype: int64


Data Augmentation(Word Embedding Augmentation (fastText))